In [0]:
!pip3 install -q tflearn
import numpy as np
from scipy.misc import imread, imresize
from glob import glob #helps find files in a folder
import os
from google.colab import drive
import matplotlib.pyplot as plt #need to plot changes
import os, sys
import glob
from scipy.misc import imread, imresize, imshow
from tflearn.layers.core import input_data
from tflearn.layers.conv import global_avg_pool
from tflearn.layers.core import fully_connected
from tensorboard import *
from tflearn.layers.estimator import regression

In [0]:
drive.mount('/content/drive') #mount drive to this directory

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#navigate to this directory
os.chdir('drive/My Drive/DeepLearningFall2018')

In [0]:
os.listdir() #list all files

['Colab_Intro.ipynb',
 'Basic_Operations.ipynb',
 'turing.jpeg',
 'Python_Numpy_Intro.ipynb',
 'DeepLearningFall2018-master.zip',
 'linear_data.csv',
 'DeepLearning_Fall2018_HW1.ipynb',
 'perceptrons_example_GalenHolland.ipynb',
 'Perceptrons_Example_GalenHolland.ipynb',
 'Midterm_Practice_TeachersEdition.ipynb',
 "Midterm_Practice2_teacher's_edition (1).ipynb",
 'multilayer_perceptron_galenholland.ipynb',
 'Midterm_Exam_Galen_Holland.ipynb',
 'Data',
 'residual_block.py',
 '__pycache__',
 'residual_block.ipynb',
 'mnist',
 'CNN_GalenHolland.ipynb',
 'ResidualBlockGalenHolland.ipynb',
 'art',
 'GaN.ipynb',
 'PoolDetector_Galen_Holland.ipynb']

In [0]:
from residual_block import *

In [0]:
def img_reader(dir, imsz, file_ex=None, sort=False):
    ''' A function that reads in images contained
        in each folder in dir, resizes them to
        imsz[0] x imsz[1], and attaches a label to
        each image based on the folder it came from.

        Args:
             dir: directory where the image folders are
                  located.
             imsz: a list of length 2, where each number
                   represents the height and width each
                   image will be resized to. If all images
                   are the same size and don't need to be
                   resized, just put the size they already
                   are.
              file_ex: if the images you want to compile
                       are all of the same extension and/or
                       there are other file types in the
                       folders that you don't want to read,
                       you can include the extension as file_ex.
                       Example, file_ex = '.jpg'
    '''
    
    main_dir = os.getcwd()
    
    if file_ex is None:
        file_ex = '*'
    else:
        file_ex = '*' + file_ex

    folders = os.listdir(dir)
    os.chdir(dir)
    num_ims=sum([len(files) for r, d, files in os.walk(os.getcwd())])
    imgs = np.zeros([num_ims, imsz[0], imsz[1], 3])
    labels = np.zeros([num_ims, 1])

    for subdir, folder in enumerate(folders):
        os.chdir(dir + '/' + folder)
        files = glob.glob(file_ex)
        if sort:
            files.sort()

        for i, file in enumerate(files):
            if os.path.isfile(file):
                img = imread(file)
                img = imresize(img, [imsz[0], imsz[1]])
                imgs[i, ...] = img
                labels[i, 0] = subdir

    os.chdir(main_dir)            
    return imgs, labels

In [0]:
os.chdir('Data') # go into Data folder where my two folders are with the images

In [0]:
os.listdir()
os.getcwd()

'/content/drive/My Drive/DeepLearningFall2018/Data'

In [0]:
#X, Y = img_reader(os.getcwd(), [150, 150])  # get images and labels as X and Y

In [0]:
import h5py
#h5file = h5py.File('data.h5', 'a')
#h5file.create_dataset('X', data=X)
#h5file.create_dataset('Y', data=Y)
#h5file.close()

In [0]:
h5file = h5py.File('data.h5', 'r')
X = np.asarray(h5file['X'])
Y = np.asarray(h5file['Y'])
h5file.close()

In [0]:
print(Y.shape)

(1084, 2)


In [0]:
import tensorflow as tf
tf.reset_default_graph()
!pip install tensorboardcolab
from tensorboardcolab import *
tcb = TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
http://7cfdf0f3.ngrok.io


In [0]:
# create the input layer for the network
input_layer = input_data([None, 150, 150, 3])

# first convolution --- 75 x 75 x 64
conv1 = conv_2d(input_layer, 64, 7, activation='relu')

# max pooling --- 37 x 37 x 64
pool1 = max_pool_2d(conv1, 3, 2)

# residual blocks 1-3 --- 37 x 37 x 64
residual_block1 = residual_block(pool1, # input to this layer
                                 64, # number of features to look for
                                 3, # filter size
                                 3) # number of residual blocks

# residual blocks 4-7 --- 18 x 18 x 128
residual_block2 = residual_block(residual_block1, # input 
                                 128, # num. features to look for
                                 3, # filter size
                                 4, # number of blocks
                                 2) # stride length 

# residual blocks 8-13 --- 9 x 9 x 256
residual_block3 = residual_block(residual_block2,
                                 256,
                                 3,
                                 6, 
                                 2)

# residual blocks 14-16 --- 4 x 4 x 512
residual_block4 = residual_block(residual_block3,
                                 512,
                                 3,
                                 3,
                                 2)

# global average pooling --- 512
gap = global_avg_pool(residual_block4)

# output layer
output_layer = fully_connected(gap, 2, activation='softmax')

In [0]:
# describe optimization 
optimizer = regression(output_layer, optimizer='adam', learning_rate=0.0001)

In [0]:
# build the model
model = tflearn.DNN(optimizer, tensorboard_verbose=3, tensorboard_dir='./Graph')

In [0]:
# training
model.fit(X,
          Y,
          batch_size=32,
          n_epoch=100,
          validation_set=0.2,
          show_metric=True,
          snapshot_step=100,
          run_id='pool')

Training Step: 7  | total loss: 7.02480 | time: 66.827s
| Adam | epoch: 001 | loss: 7.02480 - acc: 0.5437 -- iter: 224/867


KeyboardInterrupt: ignored

In [0]:
model.save('pool_detector')

INFO:tensorflow:/content/drive/My Drive/DeepLearningFall2018/Data/pool_detector is not in all_model_checkpoint_paths. Manually adding it.
